In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.models import load_model

from src.nn_settings.gru_tflite_config import Config
from src.tflite_rnn import convertible_gru #, gru_functional, lstm_functional, tflite_lstm

/Users/i862304/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
keras_file = "../data/keras_rnn_1.h5"

config = Config()
data = config.data
num_input_characters = config.num_input_characters
num_output_characters = config.num_output_characters
input_encoding = config.input_encoding
output_encoding = config.output_encoding
enc = config.encoder
max_input_length = enc.max_input_length
max_output_length = enc.max_output_length

print("data_size", config.data_config.data_size)
print("char_string", config.data_config.char_string)

batch_size = config.batch_size
epochs = config.epochs
latent_dim = config.latent_dim

print("batch_size", batch_size)
print("epochs", epochs)
print("latent_dim", latent_dim)

print("\ninput chars", num_input_characters)
print("data size", len(data))
print(data[:5])



Creating Data set


/Users/i862304/workspace/NeuralNetworkPlayground/notebooks/src/dataset_builders/text_data.py:59: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  sample_data = np.vstack((get_one_sample() for _ in range(data_size)))


Building Vocabulary
Building Encoding
data_size 30000
char_string abcdeXY
batch_size 32
epochs 4
latent_dim 8

input chars 9
data size 30000
[['caXbcYbd' 'bc']
 ['cdXbcYbe' 'bc']
 ['ccXecYcc' 'ec']
 ['bbXccYce' 'cc']
 ['ecXebYae' 'eb']]


In [4]:
# from tensorflow.python.keras.models import Model, Sequential
# from tensorflow.python.keras.layers import Input, GRU, Dense, LSTM
# from tensorflow.python.keras.regularizers import l2
# from tensorflow.python.keras.layers import TimeDistributed, Bidirectional, RepeatVector
# from tensorflow.python.keras.optimizers import RMSprop

# def temp_gru(input_size, 
#              output_size,
#              latent_dim,
#              max_input_length, 
#              max_out_seq_len,
#              use_bidirectional=False,
#             ):
#     """
#     A simple GRU stripped down about as much as possible
#     """
#     inputs = Input(shape=(max_input_length, input_size,))
#     if use_bidirectional:
#         x = Bidirectional(
#                 GRU(latent_dim,
#                     return_sequences=False,
#                     unroll=True,
#                 ),
#                 merge_mode='concat',
#             )(inputs)
#         latent_dim_2 = latent_dim * 2
#     else:
#         x = GRU(latent_dim,
#                 return_sequences=False,
#                 unroll=True,
#                )(inputs)
#         latent_dim_2 = latent_dim        
#     x = RepeatVector(max_out_seq_len)(x)
#     x = GRU(latent_dim_2, 
#             kernel_initializer='glorot_uniform', 
#             recurrent_initializer='orthogonal',  
#             return_sequences=True,
#             unroll=True,
#            )(x)
#     outputs = TimeDistributed(
#                 Dense(output_size, activation="softmax")
#               )(x)
#     model = Model(inputs, outputs)

#     rms_prop = RMSprop(
#                        rho=0.9,
#                        decay=0.0,
#                        epsilon=1e-08,
#                        )
#     model.compile(loss="categorical_crossentropy",
#                   optimizer=rms_prop)
    
#     return model


# # latent_dim = 16
# model = temp_gru(num_input_characters,
#                  num_output_characters,
#                  latent_dim,
#                  max_input_length, 
#                  max_output_length,
#                 )




In [6]:
model = convertible_gru(num_input_characters,
                 num_output_characters,
                 latent_dim,
                 max_input_length, 
                 max_output_length,
                )

# model = gru_functional(
#            num_input_characters,
#            num_output_characters,
#            latent_dim,
#            max_input_length,
#            max_output_length,
#           )

# model = lstm_functional(
#            num_input_characters,
#            num_output_characters,
#            latent_dim,
#            max_input_length,
#            max_output_length,
#           )

# model = tflite_lstm(
#            num_input_characters,
#            num_output_characters,
#            latent_dim,
#            max_input_length,
#            max_output_length,
#           )

print(model.summary())

model.fit(input_encoding, 
          output_encoding,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1)

model.save(keras_file)
# tf.keras.models.save_model(model, keras_file)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 8, 9)              0         
_________________________________________________________________
gru_2 (GRU)                  (None, 42)                6552      
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 2, 42)             0         
_________________________________________________________________
gru_3 (GRU)                  (None, 2, 42)             10710     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 2, 9)              387       
Total params: 17,649
Trainable params: 17,649
Non-trainable params: 0
_________________________________________________________________
None
Train on 27000 samples, validate on 3000 samples
Epoch 1/4
27000/27000 [==============================] - 8s 308us/sample - loss: 0.8

In [5]:
## Demonstrate that the model does something
test_sample = "acXacYda"
test_enc = enc.encode([test_sample], encode_as="input")
keras_pred = model.predict(test_enc)
print(keras_pred)
enc.decode(keras_pred, decode_as="output")[0]

[[[9.3975641e-06 7.9116026e-06 9.0582979e-01 8.2582325e-02 8.1784846e-03
   4.1360719e-05 3.3343162e-03 8.6330119e-06 7.8870062e-06]
  [1.1875411e-07 1.5408457e-07 6.8946034e-02 1.9312893e-04 9.2969137e-01
   6.5857741e-05 1.1030507e-03 1.5557679e-07 9.4823427e-08]]]


b'ac'

In [6]:
## Convert to TensorFlow Lite model.
tflite_file = "../data/tflite_rnn_1.h5"

## These docs describe the converter in the TensorFlow nightly release, installed using pip install tf-nightly
# converter = tf.lite.TFLiteConverter.from_keras_model_file(keras_file)

## For tensorflow 1.12 it should be 
converter = tf.contrib.lite.TFLiteConverter.from_keras_model_file(keras_file)

tflite_model = converter.convert()
print("Writing model data to file ...")
open(tflite_file, "wb").write(tflite_model)
print("done")


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.
Writing model data to file ...
done


In [7]:
# ss = "RuntimeError: TOCO failed see console for info.\nb'2019-02-13 14:45:21.834947: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayV3\n2019-02-13 14:45:21.835013: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1127] Op node missing output type attribute: gru/TensorArray\n2019-02-13 14:45:21.835033: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayV3\n2019-02-13 14:45:21.835050: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1127] Op node missing output type attribute: gru/TensorArray_1\n2019-02-13 14:45:21.835121: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayScatterV3\n2019-02-13 14:45:21.835139: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1127] Op node missing output type attribute: gru/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3\n2019-02-13 14:45:21.835168: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835194: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835213: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835231: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835274: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835297: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835320: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: LoopCond\n2019-02-13 14:45:21.835332: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1127] Op node missing output type attribute: gru/while/LoopCond\n2019-02-13 14:45:21.835389: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayReadV3\n2019-02-13 14:45:21.835409: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835424: I tensorflow/contrib/lite/toco/import_tensorflow.cc:189] Unsupported data type in placeholder op: 20\n2019-02-13 14:45:21.835440: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835458: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.835475: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835489: I tensorflow/contrib/lite/toco/import_tensorflow.cc:189] Unsupported data type in placeholder op: 20\n2019-02-13 14:45:21.835548: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.835602: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.835657: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.835674: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835688: I tensorflow/contrib/lite/toco/import_tensorflow.cc:189] Unsupported data type in placeholder op: 20\n2019-02-13 14:45:21.835738: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.835790: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.835842: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.835859: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.835872: I tensorflow/contrib/lite/toco/import_tensorflow.cc:189] Unsupported data type in placeholder op: 20\n2019-02-13 14:45:21.835921: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.836051: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: ReadVariableOp\n2019-02-13 14:45:21.836130: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayWriteV3\n2019-02-13 14:45:21.836145: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1127] Op node missing output type attribute: gru/while/TensorArrayWrite/TensorArrayWriteV3\n2019-02-13 14:45:21.836162: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Enter\n2019-02-13 14:45:21.836176: I tensorflow/contrib/lite/toco/import_tensorflow.cc:189] Unsupported data type in placeholder op: 20\n2019-02-13 14:45:21.836211: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Exit\n2019-02-13 14:45:21.836226: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: Exit\n2019-02-13 14:45:21.836248: I tensorflow/contrib/lite/toco/import_tensorflow.cc:1080] Converting unsupported operation: TensorArrayReadV3\n2019-02-13 14:45:21.839639: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] Before Removing unused ops: 102 operators, 178 arrays (0 quantized)\n2019-02-13 14:45:21.841691: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] Before general graph transformations: 102 operators, 178 arrays (0 quantized)\n2019-02-13 14:45:21.843801: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] After general graph transformations pass 1: 93 operators, 165 arrays (0 quantized)\n2019-02-13 14:45:21.971145: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] After general graph transformations pass 2: 92 operators, 164 arrays (0 quantized)\n2019-02-13 14:45:21.973872: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] After general graph transformations pass 3: 91 operators, 162 arrays (0 quantized)\n2019-02-13 14:45:21.975570: I tensorflow/contrib/lite/toco/graph_transformations/graph_transformations.cc:39] Before dequantization graph transformations: 91 operators, 162 arrays (0 quantized)\n2019-02-13 14:45:21.976779: F tensorflow/contrib/lite/toco/tooling_util.cc:618] Check failed: dim >= 1 (0 vs. 1)\n'"
# print(ss)

# Check performance of tflite model

In [8]:
# Load TFLite model and allocate tensors.
interpreter = tf.contrib.lite.Interpreter(model_path=tflite_file)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
# change the following line to feed into your own data.
# input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
input_data = test_enc.reshape((1,8,9))
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
tflite_pred = interpreter.get_tensor(output_details[0]['index'])
print(tflite_pred)
print(enc.decode(tflite_pred, decode_as="output")[0])

[[[5.5792561e-04 3.4225883e-04 7.4655816e-02 2.7901456e-01 2.6140375e-02
   2.8736067e-01 3.3091739e-01 4.6484091e-04 5.4622925e-04]
  [3.3376069e-04 1.8442821e-04 6.3843288e-02 2.7032682e-01 7.3219743e-03
   5.0714767e-01 1.5024687e-01 2.6324904e-04 3.3197901e-04]]]
b'ed'


In [9]:
for ii, (aa, bb) in enumerate(zip(tflite_pred.flatten(), keras_pred.flatten())):
    print("{:.9f} | {:.9f} | {:.9f}".format(aa, bb, aa-bb))
    if ii == 8:
        print("--"*20)

0.000557926 | 0.000009398 | 0.000548528
0.000342259 | 0.000007912 | 0.000334347
0.074655816 | 0.905829787 | -0.831173956
0.279014558 | 0.082582325 | 0.196432233
0.026140375 | 0.008178485 | 0.017961890
0.287360668 | 0.000041361 | 0.287319303
0.330917388 | 0.003334316 | 0.327583075
0.000464841 | 0.000008633 | 0.000456208
0.000546229 | 0.000007887 | 0.000538342
----------------------------------------
0.000333761 | 0.000000119 | 0.000333642
0.000184428 | 0.000000154 | 0.000184274
0.063843288 | 0.068946034 | -0.005102746
0.270326823 | 0.000193129 | 0.270133704
0.007321974 | 0.929691374 | -0.922369421
0.507147670 | 0.000065858 | 0.507081807
0.150246873 | 0.001103051 | 0.149143830
0.000263249 | 0.000000156 | 0.000263093
0.000331979 | 0.000000095 | 0.000331884
